# Palabras autoreferentes
En este script buscamos las palabras que ene el grafo tienen un bucle y ver su contexto, es decir mostrar su párrafo. Para lo cual modificamos el código que genera el grafo:

In [8]:
import nltk#librería para el tratamiento sintáctico
from itertools import imap,ifilter,islice,repeat#tanto imap como ifilter son iteradores, son usados
               #por razones de eficiencia ver https://docs.python.org/2/library/itertools.html
import networkx as nx
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer("spanish")
#ajustamos el sistema para tratar el utf-8:
import sys
reload(sys)
sys.setdefaultencoding('utf8')
def pairwise(iterable):
    """otro iterador:(extraido de https://docs.python.org/2/library/itertools.html)
    s -> (s0,s1), (s1,s2), (s2, s3), ...
    """
    a, b = next(iterable),next(iterable),
    while 1:
        yield (a, b)
        a,b=b,next(iterable)
endphase='.!?¿!-;.:'
import stop_words
stopwords=stop_words.get_stop_words(u'spanish')#cargamos las stopwords del idioma

In [9]:
text=open('BIBLIA.txt','r').read()
raw=str.decode( text,'utf8')
tokens = nltk.word_tokenize(raw,'spanish')
tokensLow=imap(unicode.lower,tokens)
from collections import defaultdict
root_original=defaultdict(set)
def getroot(original):
    root=spanish_stemmer.stem(original)
    root_original[root].add(original)
    return root
relevantWords=imap(getroot,ifilter(lambda x: not(x in stopwords), tokensLow) )
digits='01234566789&#()[]'
def punt_gestion(pairs):
    for (a,b) in pairs:
        if a in endphase or b in endphase or (a[0] in digits) or (b[0] in digits):
            pass
        else:
            yield (a,b)
parejas=(punt_gestion(pairwise(ifilter(lambda x: not(x in ','),relevantWords) )))

En esta sección el código se modifica para retener en un diccionario (pareja_rep_frase) las palabras autorrferenciadas$\rightarrow$ numero parrafo.

In [10]:
root_original=defaultdict(set)
line=defaultdict(list)
from itertools import imap,ifilter,islice,repeat,izip
def getroot(original,n):
    root=spanish_stemmer.stem(original)
    root_original[root].add(original)
    line[root].append(n)
    return root
def punt_gestion(pairs):
        for (a,b) in pairs:
            if a in endphase or b in endphase or (a[0] in digits) or (b[0] in digits):
                pass
            else:
                yield (a,b)
text=enumerate(open('BIBLIA.txt','r'))
pareja_rep_frase=defaultdict(set)
def app(set,(pairword,numparr) ):
    if pairword[0]==pairword[1]:
        set[pairword].add(numparr)
    return set
for numparr,parr in text:
    raw=str.decode( parr,'utf8')
    tokens = nltk.word_tokenize(raw,'spanish')
    tokensLow=imap(unicode.lower,tokens)
    relevantWords=imap(getroot,ifilter(lambda x: not(x in stopwords), tokensLow) ,repeat(numparr))
    digits='01234566789&#()[]'
    
    pareja_rep_frase=reduce(app,izip(punt_gestion(pairwise(ifilter(lambda x: not(x in ','),relevantWords) )),repeat(numparr)),pareja_rep_frase)
pareja_rep_frase=dict([(a,b) for a,b in pareja_rep_frase.iteritems() if len(b)>5])

In [11]:
text=list(open('BIBLIA.txt','r'))

Escribimos los resultados en [extractosRep.txt](extractosRep.txt)

In [38]:
text_=lambda x:(a,text[x])
r=[]
f=open('extractosRep.txt','w')
for i in [map(text_ ,list(b)[:2] )   for (a,b) in pareja_rep_frase.items()]:#[[((word,word),)]]
    for j in i:
        ((w,w),xs)=j
        r.append(w)
        f.write('\n\n')
        f.write(w+':\n\n')
        f.write((xs))
f.close()